In [ ]:
model_name = "qwen2.5:7b" # or other models to test
baseline_name = "roleplay" # or set other baselines / methods
input_folder = f'../method_outputs/{baseline_name}/{model_name}'
output_folder = f'../judge_outputs/granite/{baseline_name}/{model_name}'

In [ ]:
import openai

client = openai.OpenAI(
    base_url='http://localhost:11434/v1',
    api_key='ollama'  
)


In [ ]:
def run(input_txt):

    response = client.chat.completions.create(
        model="granite3-guardian:8b",
        messages=[
            {"role": "system", "content": ''},
            {"role": "user", "content": input_txt}
        ]
    )

    res = response.choices[0].message.content

    
    return res.lower()

In [ ]:
import os
import json
from tqdm import tqdm

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Process each file in the input folder
for entry in tqdm(os.scandir(input_folder)):
    if entry.is_file():  # Ensures it's a file
        input_file_path = entry.path
        output_file_path = os.path.join(output_folder, entry.name)
        
        if os.path.exists(output_file_path):
            continue
        
        with open(input_file_path, 'r') as f:
            data = json.load(f)

        statement_text = data['output']
        try:
            res = run(statement_text)
        except Exception as e:
            print(f"Error processing {entry.name}")
            continue

        data['judge'] = res

        with open(output_file_path, 'w') as f:
            json.dump(data, f, indent=4)  # `indent=4` makes it pretty-printed